configuration of all the variable  cosmosdb and open AI
 The architecture use changefeed and some method to load data 
 
 Loadpdf , loadword for exemple 
 load json too informatio in the db 

 after use readfeed to generate le vector in a vector db 

 when you look for similarity it will give you the source and id of doc in the source , 

 after send the information to llm open ai and build the answer 
 


In [1]:

import config
import json
import os
import sys
import uuid
import datetime
import glob
import time
import uuid
import csv
import gradio as gr





from openai import AzureOpenAI
from azure.core.exceptions import AzureError
from azure.cosmos import CosmosClient, PartitionKey
from dotenv import dotenv_values
from azure.cosmos import ThroughputProperties
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# specify the name of the .env file name 
env_name = "example.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)
# Azure Cosmos DB connection details
HOST = config['cosmos_host']
MASTER_KEY = config['cosmos_key']

cosmos_connection_string = config['cosmos_string']

container_name = "ChatMessages"


# Azure OpenAI connection details
openai_endpoint = config['openai_endpoint']
openai_key = config['openai_key']
openai_version = config['openai_version']
openai_embeddings_model = config['openai_embeddings_deployment']
openai_chat_model = config['AZURE_OPENAI_CHAT_MODEL']

# dalle = config['Dall-e']  

dbsource = config['cosmosdbsourcedb'] 
colvector = config['cosmosdbsourcecol']
cachecol = config['cosmsodbcache']
cosmosdbcolcompletion = config['cosmosdbcolcompletion']
container_name = config['cosmosdbcolcompletion']

# Create the OpenAI client
openai_client = AzureOpenAI(
  api_key = openai_key,  
  api_version = openai_version,  
  azure_endpoint =openai_endpoint 
)


This Python function, [`createvectordb`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is designed to create a container in an Azure Cosmos DB database with specific indexing and vector embedding policies. Here's a step-by-step breakdown of its operations:

1. **Initialize Cosmos Client**: It starts by creating a [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") object using a [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a dictionary containing a `masterKey` as [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"). This client is used to interact with the Azure Cosmos DB service.

2. **Get Database Client**: The function then retrieves a client for the specific database ([`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A42%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) using the [`get_database_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A365%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") method of the [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") object.

3. **Print Collection Name**: It prints the name of the collection (container) to be created or ensured exists.

4. **Define Vector Embedding Policy**: A dictionary named [`vector_embedding_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A7%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is defined, specifying the settings for vector embeddings. This includes the path (`/embedding`), data type (`float32`), distance function (`cosine`), and dimensions (`1536`) of the embeddings.

5. **Define Indexing Policy**: Another dictionary, [`indexing_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A8%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is defined to specify which paths are included and excluded in the indexing, as well as the vector indexes configuration. It includes all paths (`/*`) except the `_etag` path and specifies a vector index on the `/embedding` path with a type of `quantizedFlat`.

6. **Create or Ensure Container Exists**: The function attempts to create a container (or ensure it exists) in the database client ([`mydbt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A3%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) with the specified `id` (collection name), partition key (on the `/id` path), offer throughput settings (with auto-scaling up to 1000 throughput units), and the previously defined indexing and vector embedding policies. If the container is successfully created or confirmed to exist, it prints "container created".

7. **Error Handling**: If an error occurs during the container creation process, the function will raise the caught exception to be handled by the caller.

This function is tailored for scenarios where there's a need to store and efficiently query large sets of vector data in Azure Cosmos DB, leveraging Cosmos DB's capabilities for NoSQL data storage and advanced indexing features like vector indexing for AI and machine learning applications.

In [4]:
def createvectordb(collection):
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)
    
    print(collection)
    
    vector_embedding_policy = { "vectorEmbeddings": [ {  "path": "/embedding",  "dataType": "float32",  "distanceFunction": "cosine",  "dimensions": 1536  } ] }
    indexing_policy = { "includedPaths": [ { "path": "/*" } ], "excludedPaths": [  {  "path": "/\"_etag\"/?" }  ], "vectorIndexes": [ {"path": "/embedding", "type": "diskANN"  }] }

    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0),
        indexing_policy=indexing_policy, 
        vector_embedding_policy=vector_embedding_policy) 
        print("container created")
              
    except : 
         raise

This code snippet demonstrates how to create a container in Azure Cosmos DB with specific indexing and vector embedding policies. Here's a step-by-step breakdown:

1. **Import Statements**:
   - `from azure.cosmos import exceptions, PartitionKey`: Imports necessary modules from the Azure Cosmos SDK.

2. **Client Initialization**:
   - [`client = CosmosClient(HOST, {'masterKey': MASTER_KEY})`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A3%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Initializes the Cosmos DB client using the provided [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A30%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A31%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

3. **Database Client**:
   - [`mydbt = client.get_database_client(dbsource)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A4%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Retrieves a database client for the specified database ([`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A45%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")).

4. **Vector Embedding Policy**:
   - [`vector_embedding_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A7%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Defines a policy for vector embeddings, specifying the path, data type, distance function, and dimensions.

5. **Indexing Policy**:
   - [`indexing_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A19%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Defines an indexing policy for vector indexes, specifying the path and type of index.

6. **Container Creation**:
   - The code attempts to create a container ([`cache_container`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A31%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) with the specified `id`, `partition_key`, [`indexing_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A19%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`vector_embedding_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A7%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and `offer_throughput`.
   - [`cache_container = mydbt.create_container_if_not_exists(...)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A31%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Creates the container if it does not already exist.
   - [`print('Container with id \'{0}\' created'.format(id))`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Fc%253A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A112%2C%22character%22%3A8%7D%5D "../../.vscode/extensions/ms-python.vscode-pylance-2024.8.1/dist/typeshed-fallback/stdlib/builtins.pyi"): Prints a success message if the container is created.

7. **Exception Handling**:
   - [`except exceptions.CosmosHttpResponseError`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fexceptions.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D ".venv/Lib/site-packages/azure/cosmos/exceptions.py"): Catches any [`CosmosHttpResponseError`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fexceptions.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A32%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/exceptions.py") exceptions that occur during container creation and raises them.

This code is designed to set up a Cosmos DB container with advanced configurations for handling vector embeddings and indexing, which can be useful for applications involving machine learning or search functionalities.

In [31]:
from azure.cosmos import exceptions, PartitionKey


client = CosmosClient(HOST, {'masterKey': MASTER_KEY})

mydbt = client.create_database_if_not_exists(dbsource)

mydbt = client.get_database_client(dbsource)
    
# Create the vector embedding policy to specify vector details
vector_embedding_policy = {
    "vectorEmbeddings": [ 
        { 
            "path":"/vector" ,
             "dataType":"float32",
            "distanceFunction":"dotproduct",
            "dimensions":1536
        }, 
    ]
}

# Create the vector index policy to specify vector details
indexing_policy = {
    "vectorIndexes": [ 
        {
            "path": "/vector", 
            "type": "diskANN" 
        }
    ]
} 





# Create the cache collection with vector index
try:
    cache_container =  mydbt.create_container_if_not_exists( id=cachecol, 
                                                  partition_key=PartitionKey(path='/id'), 
                                                  indexing_policy=indexing_policy,
                                                  vector_embedding_policy=vector_embedding_policy,
                                                  offer_throughput=1000) 
    print('Container with id \'{0}\' created'.format(id)) 

except exceptions.CosmosHttpResponseError: 
        raise 

Container with id '<built-in function id>' created


This function, [`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), takes two parameters: [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A24%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`text`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"). It is designed to generate embeddings for the given [`text`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") using the OpenAI API version 1.x. Here's a step-by-step description of what it does:

1. It calls the `embeddings.create` method on the [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A24%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") object. This method is passed two arguments:
   - `input`: The text for which embeddings are to be generated.
   - `model`: A variable [`openai_embeddings_model`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A39%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") that presumably specifies the model to be used for generating embeddings. This variable is not defined within the function, so it must be defined elsewhere in the code or passed into the function's scope in some other way.

2. The response from the `embeddings.create` method is expected to be an object that contains the generated embeddings within its `data` attribute. The code assumes that the response's `data` attribute is a list, and it accesses the first item of this list with `[0]`.

3. From this first item, it accesses the `embedding` attribute to retrieve the actual embeddings.

4. Finally, the function returns the retrieved embeddings.

This function is specifically tailored for use with the OpenAI API and requires an [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A24%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") object that is properly authenticated and configured to interact with the OpenAI API. The function assumes that the API's response is structured in a particular way, which is important to note for maintenance and debugging purposes.

In [6]:
def generate_embeddings(openai_client, text):
    """
    Generates embeddings for a given text using the OpenAI API v1.x
    """
    
    print("Generating embeddings for: ", text, " with model: ", openai_embeddings_model)
    response = openai_client.embeddings.create(
        input = text,
        model= openai_embeddings_model
        
    )
    
    embeddings = response.data[0].embedding
    return embeddings





This Python function, [`loaddata`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is designed to interact with an Azure Cosmos DB instance. It performs several operations as outlined below:

1. **Initialization**: It takes three parameters: [`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A13%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the database name), [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A16%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the collection or container name within the database), and [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22external%22%3A%22vscode-notebook-cell%3A%2Fc%253A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%23W6sZmlsZQ%253D%253D%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the path to a file containing data to be loaded into the database).

2. **Client Setup**: It initializes a CosmosClient object using a [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), which are presumably defined elsewhere in the code. This client is used to interact with the Cosmos DB service.

3. **Database and Container Access**: It obtains a reference to the specified database ([`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A13%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) and attempts to create a new container within this database if one does not already exist with the specified [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A16%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") name. The container is configured with a partition key on the path `/id` and auto-scalable throughput settings (starting with a maximum throughput of 1000 RU/s and an unspecified increment percentage).

4. **Data Loading**:
   - Opens the specified [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22external%22%3A%22vscode-notebook-cell%3A%2Fc%253A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%23W6sZmlsZQ%253D%253D%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (assumed to be in JSON format) and reads its contents.
   - Iterates over the documents (assumed to be a list of JSON objects) in the file.
   - For each document, it upserts (updates or inserts) the document into the container. This means if the document does not exist, it is inserted, and if it does exist (based on some unique identifier, presumably the `/id`), it is updated.

5. **Counting Documents**:
   - Executes a query against the container to count the total number of documents.
   - Iterates over the query results (which should return a single count value) and sums up the counts to get [`total_count`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A24%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
   - Prints the total count of documents in the container.

6. **Error Handling**: The function uses a broad `try-except` block to catch any exceptions that occur during its execution. If an exception is caught, it simply re-raises the exception without handling it.

This function demonstrates how to interact with Azure Cosmos DB using the CosmosClient from the Azure SDK for Python, including creating containers, upserting documents, and querying data within a container.

In [7]:
def loaddata(db,collection,file) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
  
    print("Creating container..."   )
    


    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
         offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0))
        
        filename = file
        with open(filename,encoding="utf8") as file:
            docu = json.load(file)
            for d in docu:
                d['text']= json.dumps(d)
                container.upsert_item(d)
        
# count products
        query = "SELECT VALUE COUNT(1) FROM c"

        total_count = 0

        result = container.query_items(
            query=query,
            enable_cross_partition_query=True)

        for item in result:
            total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  
       

  

This Python function, [`loadpdffile`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), performs several operations to load text data from a PDF file into a Cosmos DB collection. Here's a step-by-step breakdown of its functionality:

1. **Initialize Cosmos Client**: It starts by creating a [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") instance using a [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a dictionary containing a `masterKey` (presumably for authentication). This client is used to interact with the Cosmos DB.

2. **Get Database Client**: The function then retrieves a client for the specified database ([`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A16%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) using the [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") instance.

3. **Load PDF Data**: It uses a [`PyPDFLoader`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fdocument_loaders%2Fpdf.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A162%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_community/document_loaders/pdf.py") (a custom or third-party class not defined in the snippet) to load data from the specified [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A35%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"). The loaded data is assumed to be text extracted from the PDF.

4. **Split Text Data**: The text data is then split into smaller documents using a [`RecursiveCharacterTextSplitter`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_text_splitters%2Fcharacter.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A57%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_text_splitters/character.py"), which likely breaks the text into chunks of 1000 characters with an overlap of 150 characters between chunks. This is useful for processing large texts or for fitting into database constraints.

5. **Print First Document**: It prints the first document chunk to the console, likely for debugging or verification purposes.

6. **Create or Get Container**: The function attempts to create a new container in the database if it doesn't exist. The container is identified by [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A19%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), uses `/id` as the partition key, and is configured with auto-scalable throughput properties.

7. **Insert Documents into Container**: For each document chunk created earlier, it generates a new document with a unique `id` (using [`uuid4`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FLib%2Fuuid.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A722%2C%22character%22%3A4%7D%5D "../../Lib/uuid.py")), the original [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A35%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") name, and the chunked `text`. Each document is then upserted into the container. Upserting means that the document will be inserted if it doesn't exist or updated if it does.

8. **Error Handling**: The insertion process is wrapped in a try-except block to handle potential errors gracefully. If an error occurs, it raises an exception.

9. **Count Documents**: After inserting the documents, it attempts to count the total number of documents in the container using a SQL query. The total count is printed to the console.

10. **Error Handling for Counting**: The counting process is also wrapped in a try-except block to handle potential errors.

This function is designed to process and store large text data from PDF files into Cosmos DB, making the data queryable and manageable. It includes basic error handling and prints out useful information for debugging.

In [8]:
def loadpdffile(db,collection,name,file) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
    
    loader = PyPDFLoader(file)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(data)
   
    print(docs[0]) 
    
    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0))
        
       
        for d in docs : 
            docu= {} 
            docu["id"] = str(uuid.uuid4())
            docu["file"] = name
            docu["text"] = str(d)
            print(docu)
            container.upsert_item(docu)
    except : 
     raise     
# count 
    try : 
     query = "SELECT VALUE COUNT(1) FROM c"

     total_count = 0

     result = container.query_items(
     query=query,
     enable_cross_partition_query=True)

     for item in result:
        total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  
       

This Python function, [`loadwordfile`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), performs several operations to load a Word document into a Cosmos DB NoSQL database. Here's a step-by-step breakdown of its functionality:

1. **Initialization**: The function takes four parameters: [`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A17%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (database name), [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (collection name within the database), [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A31%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (a name identifier for the file), and [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A36%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the path to the Word document file).

2. **CosmosClient Setup**: It initializes a CosmosClient using a predefined [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to connect to the Azure Cosmos DB instance.

3. **Database Client**: Retrieves a client for the specified database ([`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A17%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) using the CosmosClient instance.

4. **Document Loader**: Utilizes the [`Docx2txtLoader`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fdocument_loaders%2Fword_document.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A16%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_community/document_loaders/word_document.py") class from the [`langchain_community.document_loaders`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2F__init__.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D ".venv/Lib/site-packages/langchain_community/__init__.py") module to load the content of the Word document specified by the [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A36%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") parameter.

5. **Text Splitting**: Employs a [`RecursiveCharacterTextSplitter`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_text_splitters%2Fcharacter.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A57%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_text_splitters/character.py") (not imported in the provided code, so assumed to be defined elsewhere or imported implicitly) to split the loaded document text into smaller chunks. This is likely for easier storage and management in the database. The splitter uses a `chunk_size` of 1000 characters and an `overlap` of 150 characters.

6. **Container Creation**: Attempts to create a new container in the database if it does not already exist, using the specified [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") name. It sets a partition key on the `/id` path and configures auto-scaling throughput properties.

7. **Document Insertion**: Iterates over the chunks of text produced by the text splitter. For each chunk, it creates a new document with a unique `id` (generated using [`uuid.uuid4()`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FLib%2Fuuid.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D "../../Lib/uuid.py")), the [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A31%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") of the file, and the chunk of text itself. Each document is then upserted into the Cosmos DB container, meaning it will be inserted if it doesn't exist or updated if it does.

8. **Count Query**: After inserting the documents, it executes a query to count the total number of documents in the


In [ ]:
def loadexcelfile(db,collection,name,file) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
    
    from langchain_community.document_loaders import Docx2txtLoader

    loader = Docx2txtLoader(file)

    data = loader.load()    
    
    
    
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(data)
   
    print(docs[0]) 
    
    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0))
        
       
        for d in docs : 
            docu= {} 
            docu["id"] = str(uuid.uuid4())
            docu["file"] = name
            docu["text"] = str(d)
            print(docu)
            container.upsert_item(docu)
    except : 
     raise     
# count 
    try : 
     query = "SELECT VALUE COUNT(1) FROM c"

     total_count = 0

     result = container.query_items(
     query=query,
     enable_cross_partition_query=True)

     for item in result:
        total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  

In [26]:
def loadcsvfile(db,collection,name,file) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
   
    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0))
        
 
    # Read CSV file and convert to JSON
        with open(file, mode='r', encoding='utf-8-sig') as file:
         csv_reader = csv.DictReader(file)
         for row in csv_reader:
                row["id"] = str(uuid.uuid4())
                row["file"] = name
                json_data = json.dumps(row)
                # Insert JSON data into Cosmos DB
                container.create_item(body=json.loads(json_data))
    
    except : 
        raise


   
# count 
    try : 
     query = "SELECT VALUE COUNT(1) FROM c"

     total_count = 0

     result = container.query_items(
     query=query,
     enable_cross_partition_query=True)

     for item in result:
        total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  

In [10]:
def loadwordfile(db,collection,name,file) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
    
    from langchain_community.document_loaders import Docx2txtLoader

    loader = Docx2txtLoader(file)

    data = loader.load()    
    
    
    
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(data)
   
    print(docs[0]) 
    
    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0))
        
       
        for d in docs : 
            docu= {} 
            docu["id"] = str(uuid.uuid4())
            docu["file"] = name
            docu["text"] = str(d)
            print(docu)
            container.upsert_item(docu)
    except : 
     raise     
# count 
    try : 
     query = "SELECT VALUE COUNT(1) FROM c"

     total_count = 0

     result = container.query_items(
     query=query,
     enable_cross_partition_query=True)

     for item in result:
        total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  
       

The function [`add_doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is designed to add a document to a specified collection in a NoSQL database, presumably in a Cosmos DB context, given its naming convention. It takes four parameters:

1. [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): This parameter is likely an instance of a client connected to the OpenAI API, used for generating embeddings.
2. [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): This represents the collection within the database where the document will be added or updated.
3. [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): A dictionary representing the document to be added. It must at least contain an "id" key.
4. [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A43%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): A string representing the source of the document, which will be added to the document under the "source" key.

The function performs the following steps:

1. Initializes an empty dictionary [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
2. Copies the "id" from the input [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
3. Adds a "source" field to [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), with its value set to the [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A43%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") parameter.
4. Calls a function [`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") with the [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a JSON string representation of [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to generate embeddings. The result is added to [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") under the "embedding" key.
5. Prints the "id" of the document to the console.
6. Attempts to add or update the document in the specified [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") by calling `upsert_item` with [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
7. Catches and prints any exceptions that occur during the process.

This function is useful for processing and storing documents with generated embeddings in a NoSQL database, with error handling to catch and report any issues during the operation.

In [11]:
def add_doc(openai_client, collection, doc,name):
   
  
  
    try:
        doc1 = {}
        doc1["id"] = doc["id"]
        doc1["source"]= name
        doc1["embedding"] = generate_embeddings(openai_client, json.dumps(doc))
        
        print(doc["id"])
        
        collection.upsert_item(doc1)
       
    except Exception as e:
        print(str(e))

This Python function, [`add_doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is designed to add a document to a specified collection in a NoSQL database, such as Azure Cosmos DB. It takes four parameters:

1. [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): An instance or client object that interacts with the OpenAI API, presumably used here for generating embeddings.
2. [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The target collection within the NoSQL database where the document will be added or updated.
3. [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The document to be added or updated in the collection. It is expected to be a dictionary with at least an "id" key.
4. [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A43%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): A string that seems to be used as a source identifier for the document.

The function performs the following steps:

- Initializes an empty dictionary [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
- Copies the "id" from the input [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
- Sets the "source" field in [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to the provided [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A43%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
- Calls a function [`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") with the [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a JSON string representation of [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to generate embeddings. The result is assigned to the "embedding" field in [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
- Prints the "id" of the document to the console.
- Attempts to add or update [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") in the specified [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") using the `upsert_item` method, which inserts the item if it does not exist or updates it if it does.
- Catches and prints any exceptions that occur during the process.

This function is useful for processing and storing documents with generated embeddings in a NoSQL database, leveraging the capabilities of an AI model (via [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) for generating those embeddings.

In [12]:

def get_completion(openai_client, model, prompt: str):    
   
    print("Prompt: ", prompt)
   
    response = openai_client.chat.completions.create(
        model = model,
        messages =   prompt,
        temperature = 0.1
    )   
   
    return response.model_dump()


    

This Python function, [`get_similar_docs`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is designed to retrieve documents from a Cosmos DB for NoSQL that are similar to a given query text. Here's a step-by-step breakdown of its operations:

1. **Function Signature**: The function accepts four parameters:
   - [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A21%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): An instance of a client connected to OpenAI, presumably used for generating embeddings.
   - [`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A36%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The name of the database within Cosmos DB.
   - [`query_text`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A40%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The user's query as a string.
   - [`limit`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A52%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): An optional parameter specifying the maximum number of documents to return, with a default value of 3.

2. **Vectorize the Question**: It uses the [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A21%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to generate embeddings for the [`query_text`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A40%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"). This process converts the query text into a vector representation that can be used for similarity comparisons.

3. **Query Cosmos DB**:
   - It constructs a SQL-like query to select the top [`limit`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A52%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") documents from a container within the database. These documents are selected based on their similarity to the query vector, calculated using a `VectorDistance` function.
   - The query is executed against the Cosmos DB using the [`query_items`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcontainer.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A365%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/azure/cosmos/container.py") method of a container client. This method is configured to allow cross-partition queries.

4. **Filter Results Based on Similarity**: The function iterates over the query results and filters out documents based on a predefined similarity threshold ([`sim = 0.78`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A16%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")). Only documents with a similarity score higher than this threshold are considered.

5. **Retrieve Full Documents**: For each document that passes the similarity threshold, the function constructs a new query to retrieve the full document from the database based on its ID.

6. **Return Results**: The function returns a list of documents that match the query criteria and exceed the similarity threshold. If no documents meet these criteria, an empty list is returned.

**Key Operations**:
- Vectorization of query text for similarity comparison.
- Execution of a Cosmos DB query to find similar documents based on vector similarity.
- Filtering of results based on a similarity threshold.
- Retrieval of full documents from the database.

**Potential Improvements**:
- The function assumes the existence of certain global variables ([`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`colvector`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A43%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) and functions ([`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) without them being passed as parameters or defined within the function, which could limit reusability and clarity.
- Error handling is not explicitly addressed, which could lead to issues if the database queries fail or if no documents meet the similarity threshold.
- The function's efficiency could potentially be improved by optimizing the database queries or the process of filtering and retrieving documents.

In [13]:
def get_similar_docs(openai_client, db, query_text, limit):
    """ 
        Get similar documents from Cosmos DB for NoSQL 

        input: 
            container: name of the container
            query_text: user question
            limit: max number of documents to return
        output:
            documents: json documents similar to the user question
            elapsed_time
    """
    # vectorize the question
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
    cvector = mydbt.get_container_client(colvector)
    sim =0.78
   
    query_vector = generate_embeddings(openai_client, query_text)
  
     
     
    query = f"""
        SELECT TOP @num_results  c.id,c.source, VectorDistance(c.embedding, @embedding) as SimilarityScore 
        FROM c
        WHERE VectorDistance(c.embedding,@embedding) > @similarity_score
        ORDER BY VectorDistance(c.embedding,@embedding)
    """
    results = cvector.query_items(
        query=query,
         parameters=[
            {"name": "@embedding", "value": query_vector},
            {"name": "@num_results", "value": limit},
            {"name": "@similarity_score", "value": sim}
        ],
        enable_cross_partition_query=True, populate_query_metrics=True
    )   
    
           
    listid = []
    source = ""
    # get products from list of id
    id_list = [id for id in results]

    for i in id_list:
            listid.append(i['id'])
            source = (i['source'])
                                  
            
        
        
 
    
    if listid == []:
        products = []
    else : 
        id_list_str = ', '.join([f"'{id}'" for id in listid]) 
        
      
        mycolt = mydbt.get_container_client(source)
            
        query = f"""
                    SELECT * FROM c 
                    WHERE  c.id IN ({id_list_str})
                """
                
            
        results = mycolt.query_items(
                    query=query,
                    enable_cross_partition_query=True
        )

        products = []
        for product in results:
            products.append(product)    

          
    

    return products
            


    


The [`ReadFeed`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function performs the following operations:

1. **Initialize Cosmos Client**: It creates a CosmosClient instance using a host URL ([`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) and a master key ([`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")). This client is used to interact with the Azure Cosmos DB service.

2. **Get Database and Container Clients**: It retrieves clients for a specific database ([`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A42%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) and two containers within that database. The first container client is for the container specified by the [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A13%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") parameter passed to the function. The second container client is for a container named "vector".

3. **Define Start Time for Reading the Feed**: It calculates a start time for reading the change feed by subtracting one day from the current datetime. This means it will read changes that occurred from one day ago up to the present.

4. **Print Start Times**: It prints the current time and the calculated start time (one day ago) to the console.

5. **Query the Change Feed**: It queries the change feed of the specified collection ([`mycolt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A7%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) starting from the calculated start time. The change feed includes changes to the items in the container, allowing applications to process or react to the data changes.

6. **Process Each Document in the Change Feed**: For each document in the change feed, it calls the [`add_doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function, passing in an [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A48%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), the container client for the "vector" container ([`mycoltembed`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A8%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")), the document itself, and the name of the collection. This suggests that [`add_doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is likely responsible for processing or storing each document, possibly involving some interaction with an OpenAI client.

In summary, the [`ReadFeed`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function is designed to connect to an Azure Cosmos DB instance, read the change feed for a specified container starting from one day ago, and process each document in the feed, potentially involving some operation with OpenAI.

In [14]:
def ReadFeed(collection):
        
       
        
        
        client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
        mydbt = client.get_database_client(dbsource)   
        mycolt = mydbt.get_container_client(collection)
        mycoltembed = mydbt.get_container_client("vector") 
        name = collection
        
     
        # Define a point in time to start reading the feed from
        time = datetime.datetime.now()
        
        print (time)
        time = time - datetime.timedelta(days=1)
        print (time)
        
           
        
        response = mycolt.query_items_change_feed(start_time=time)
        #response = mycolt.query_items_change_feed( )
        
        for doc in response:
            add_doc(openai_client, mycoltembed, doc,name)
           
            
            
        
 

 

The function [`get_chat_history`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X46sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") retrieves a specified number of recent chat history items from an Azure Cosmos DB container. Here's a step-by-step breakdown:

1. **Function Definition**: 
   - The function [`get_chat_history`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X46sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") takes two parameters:
     - [`container`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X46sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A21%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The Cosmos DB container from which to query items.
     - [`completions`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X46sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A32%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The number of recent chat history items to retrieve (default is 3).

2. **Query Execution**:
   - The function executes a SQL query on the [`container`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X46sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A21%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to select the top `@completions` items ordered by their timestamp (`_ts`) in descending order.
   - The query uses a parameter `@completions` to dynamically set the number of items to retrieve.

3. **Parameters**:
   - The parameter `@completions` is set to the value of the [`completions`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X46sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A32%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") argument passed to the function.

4. **Cross-Partition Query**:
   - The query is enabled to run across multiple partitions in the Cosmos DB container.

5. **Results Conversion**:
   - The results of the query are converted to a list.

6. **Return**:
   - The function returns the list of results.

### Python Code


In [15]:
def get_chat_history( completions=3):
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)   
    container = mydbt.get_container_client(cachecol)
    
    results = container.query_items(
        query= '''
        SELECT TOP @completions *
        FROM c
        ORDER BY c._ts DESC
        ''',
        parameters=[
            {"name": "@completions", "value": completions},
        ], enable_cross_partition_query=True)
    results = list(results)
    return results

The provided code defines a function [`cache_search`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") that queries a Cosmos DB container for items based on vector similarity. Here's a step-by-step breakdown:

1. **Function Definition**: 
   - [`cache_search`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") takes three parameters:
     - [`vectors`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A18%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The vector to compare against.
     - [`similarity_score`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The threshold for similarity (default is 0.99).
     - [`num_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A53%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The number of results to return (default is 5).

2. **Cosmos DB Client Initialization**:
   - A [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") is created using [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A30%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A31%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
   - The database client is retrieved using [`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A45%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
   - The container client is retrieved using [`cachecol`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A47%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

3. **Query Execution**:
   - The function executes a SQL-like query on the container:
     - Selects the top [`num_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A53%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") items.
     - Computes the similarity score using `VectorDistance`.
     - Filters items where the similarity score is greater than [`similarity_score`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
     - Orders the results by similarity score.

4. **Query Parameters**:
   - The query uses parameters for embedding vector, number of results, and similarity score.

5. **Query Options**:
   - `enable_cross_partition_query` is set to `True` to allow querying across partitions.
   - `populate_query_metrics` is set to `True` to gather query metrics.

6. **Processing Results**:
   - An empty list [`formatted_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A20%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is initialized.
   - The function iterates over the query results, prints each result, and appends it to [`formatted_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A20%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

7. **Return Statement**:
   - The function returns the [`formatted_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A20%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") list.

This function is designed to search for items in a Cosmos DB container based on vector similarity and return the top results that meet the specified criteria.

In [16]:
def cache_search( vectors, similarity_score = 0.99 , num_results=5):
    # Execute the query
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)   
    container = mydbt.get_container_client(cachecol)
    
    results = container.query_items(
        query= '''
        SELECT TOP @num_results  c.completion, VectorDistance(c.vector, @embedding) as SimilarityScore 
        FROM c
        WHERE VectorDistance(c.vector,@embedding) > @similarity_score 
        ORDER BY VectorDistance(c.vector,@embedding)
        ''',
        parameters=[
            {"name": "@embedding", "value": vectors},
            {"name": "@num_results", "value": num_results},
            {"name": "@similarity_score", "value": similarity_score}
        ],
        enable_cross_partition_query=True, populate_query_metrics=True)
   
    formatted_results = []
    for result in results:
        print("result query")
        print(result)
        formatted_results.append(result)

  
    return formatted_results

This code defines a function [`cacheresponse`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") that caches a response from a user prompt into an Azure Cosmos DB NoSQL database. Here's a step-by-step breakdown:

1. **Function Definition**: The function [`cacheresponse`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") takes three parameters: [`user_prompt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A18%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`prompt_vectors`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A31%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and [`response`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A47%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

2. **Cosmos DB Client Initialization**:
    - [`client = CosmosClient(HOST, {'masterKey': MASTER_KEY})`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A2%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Initializes a Cosmos DB client using the provided [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A30%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A31%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
    - [`mydbt = client.get_database_client(dbsource)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A3%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Retrieves a database client for the specified database ([`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A45%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")).
    - [`container = mydbt.get_container_client(cachecol)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A4%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Retrieves a container client for the specified container ([`cachecol`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A47%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")).

3. **Logging**:
    - [`print("Caching response for prompt: ", user_prompt)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Fc%253A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A112%2C%22character%22%3A8%7D%5D "../../.vscode/extensions/ms-python.vscode-pylance-2024.8.1/dist/typeshed-fallback/stdlib/builtins.pyi"): Logs the user prompt.
    - [`print("Response: ", response)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Fc%253A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A112%2C%22character%22%3A8%7D%5D "../../.vscode/extensions/ms-python.vscode-pylance-2024.8.1/dist/typeshed-fallback/stdlib/builtins.pyi"): Logs the response.

4. **Chat Document Creation**:
    - A dictionary [`chat_document`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A11%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is created to represent the chat document. It includes:
        - `'id'`: A unique identifier generated using [`uuid.uuid4()`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FLib%2Fuuid.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D "../../Lib/uuid.py").
        - `'prompt'`: The original user prompt.
        - `'completion'`: The content of the response message.
        - `'completionTokens'`, `'promptTokens'`, `'totalTokens'`: Token usage statistics from the response.
        - `'model'`: The model used to generate the response.
        - `'vector'`: The prompt vectors.

5. **Inserting Document into Cosmos DB**:
    - [`container.create_item(body=chat_document)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A4%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Inserts the [`chat_document`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A11%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") into the Cosmos DB container.
    - [`print("item inserted into cache.", chat_document)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Fc%253A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A112%2C%22character%22%3A8%7D%5D "../../.vscode/extensions/ms-python.vscode-pylance-2024.8.1/dist/typeshed-fallback/stdlib/builtins.pyi"): Logs the inserted document.

This function effectively logs and stores the interaction data into a Cosmos DB container for future reference or analysis.

In [17]:
def cacheresponse(user_prompt, prompt_vectors, response):
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)   
    container = mydbt.get_container_client(cachecol)
    
    print("Caching response for prompt: ", user_prompt)
    print("Response: ", response)
    
    
    # Create a dictionary representing the chat document
    chat_document = {
        'id':  str(uuid.uuid4()),  
        'prompt': user_prompt,
        'completion': response['choices'][0]['message']['content'],
        'completionTokens': str(response['usage']['completion_tokens']),
        'promptTokens': str(response['usage']['prompt_tokens']),
        'totalTokens': str(response['usage']['total_tokens']),
        'model': response['model'],
        'vector': prompt_vectors
    }
    # Insert the chat document into the Cosmos DB container
    container.create_item(body=chat_document)
    print("item inserted into cache.", chat_document)

The provided code defines a function [`clearcache()`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") that interacts with a Cosmos DB instance. Here's a step-by-step description:

1. **Initialize Cosmos Client**:
   - [`client = CosmosClient(HOST, {'masterKey': MASTER_KEY})`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A2%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Creates a Cosmos DB client using the provided [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A30%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A31%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

2. **Get Database Client**:
   - [`mydbt = client.get_database_client(dbsource)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A3%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Retrieves a database client for the specified [`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A45%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

3. **Define Vector Embedding Policy**:
   - [`vector_embedding_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A8%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Specifies details for vector embeddings, including the path, data type, distance function, and dimensions.

4. **Define Indexing Policy**:
   - [`indexing_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A20%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Specifies details for vector indexes, including the path and type.

5. **Delete Existing Container**:
   - [`mydbt.delete_container(cachecol)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A3%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Deletes the container specified by [`cachecol`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A47%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

6. **Create Cache Collection with Vector Index**:
   - A `try` block attempts to create a new container with the specified `id`, partition key, indexing policy, vector embedding policy, and throughput.
   - If successful, it prints a confirmation message.
   - If a [`CosmosHttpResponseError`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fexceptions.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A32%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/exceptions.py") occurs, it raises the exception.

The function essentially clears a cache by deleting an existing container and then recreating it with specific vector embedding and indexing policies.

In [18]:
def clearcache ():
   
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)   
  
    
      
# Create the vector embedding policy to specify vector details
    vector_embedding_policy = {
    "vectorEmbeddings": [ 
        { 
            "path":"/vector" ,
             "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions":1536
        }, 
    ]
}

# Create the vector index policy to specify vector details
    indexing_policy = { 
    "vectorIndexes": [ 
        {
            "path": "/vector", 
            "type": "diskANN"
            
        }
    ]
    } 
   
    mydbt.delete_container(cachecol)


# Create the cache collection with vector index
    try:
        mydbt.create_container_if_not_exists( id=cachecol, 
                                                  partition_key=PartitionKey(path='/id'), 
                                                  indexing_policy=indexing_policy,
                                                  vector_embedding_policy=vector_embedding_policy,
                                                  offer_throughput=1000) 
        print('Container with id \'{0}\' created'.format(id)) 

    except exceptions.CosmosHttpResponseError: 
        raise 
    
    
    

The provided code defines a function [`generatecompletionede`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") that generates a completion response using the OpenAI Completions API. Here's a step-by-step breakdown:

1. **Function Definition**:
   - The function [`generatecompletionede`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") takes three parameters: [`user_prompt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A26%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`vector_search_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and [`chat_history`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A62%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

2. **System Prompt**:
   - A multi-line string [`system_prompt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A2%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is defined, which sets the context for the assistant. It describes the assistant's role and behavior.

3. **Messages List**:
   - An empty list [`messages`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is initialized with a dictionary containing the [`system_prompt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A2%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") as the first message.

4. **Chat History**:
   - The function iterates over [`chat_history`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A62%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), appending each chat's prompt and completion to the [`messages`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") list as a single string.

5. **User Prompt**:
   - The [`user_prompt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A26%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is appended to the [`messages`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") list as a message from the user.

6. **Vector Search Results**:
   - The function iterates over [`vector_search_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), appending each result's text to the [`messages`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") list as a message from the system.

7. **Completion Request**:
   - The function calls [`get_completion`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X22sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A1%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") with [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A52%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`openai_chat_model`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A43%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and the [`messages`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X36sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") list to generate a response from the OpenAI API.
   - The response is printed to the console.

8. **Return Response**:
   - The function returns the response obtained from the OpenAI API.

The function essentially prepares a structured payload of messages combining system prompts, user prompts, chat history, and vector search results, and sends it to the OpenAI API to generate a completion response.

In [19]:
def generatecompletionede(user_prompt, vector_search_results, chat_history):
    
    system_prompt = '''
    You are an intelligent assistant for ypurdata . You are designed to provide helpful answers to user questions about  your data.
    You are friendly, helpful, and informative and can be lighthearted. Be concise in your responses, but still friendly.
        - Only answer questions related to the information provided below. 
        - Write two lines of whitespace between each answer in the list.
    '''

    # Create a list of messages as a payload to send to the OpenAI Completions API

    # system prompt
    
    messages = [{'role': 'system', 'content': system_prompt}]
    
    #chat history
    for chat in chat_history:
        messages.append({'role': 'user', 'content': chat['prompt'] + " " + chat['completion']})
    
    #user prompt
    messages.append({'role': 'user', 'content': user_prompt})

    #vector search results
    for result in vector_search_results:
        messages.append({'role': 'system', 'content': result['text']})

    
    # Create the completion
    response = get_completion(openai_client, openai_chat_model, messages)
    print("Response from openai", response)
    
    return response

The function [`chat_completion`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") processes a user's input to generate a response, leveraging both cached results and new computations if necessary. Here's a step-by-step breakdown:

1. **Function Definition**:
   - The function [`chat_completion`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") takes one parameter:
     - [`user_input`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The input provided by the user.

2. **Generate Embeddings**:
   - The function generates embeddings for the [`user_input`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") using the [`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function with [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A52%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`user_input`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") as arguments.

3. **Cache Search**:
   - It searches the chat history cache for similar questions using the [`cache_search`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function with the generated embeddings, a similarity score threshold of 0.99, and a request for 1 result.
   - If the cache contains results ([`len(cache_results) > 0`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Fc%253A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A1019%2C%22character%22%3A8%7D%5D "../../.vscode/extensions/ms-python.vscode-pylance-2024.8.1/dist/typeshed-fallback/stdlib/builtins.pyi")), it returns the cached completion and a boolean `True`.

4. **New Computation**:
   - If no cached results are found:
     - It prints "New result" to indicate a new computation is being performed.
     - It performs a vector search on the movie collection using the [`get_similar_docs`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X24sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function with [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A52%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W1sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A45%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`user_input`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and a request for 3 results.
     - It prints "search result done" after completing the search.

5. **Chat History**:
   - It prints "Getting Chat History" and retrieves the chat history using the [`get_chat_history`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X31sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function with a request for 3 recent items.

6. **Generate Completion**:
   - It prints "Generating completions" and generates a completion using the [`generatecompletionede`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X34sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function with [`user_input`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`search_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A16%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and [`chat_history`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A21%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") as arguments.

7. **Cache Response**:
   - It prints "Caching response" and caches the generated response using the [`cacheresponse`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X33sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function with [`user_input`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`user_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A3%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and [`completions_results`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X35sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A25%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

8. **Return**:
   - It prints a newline for separation.
   - It returns the generated completion's content and a boolean `False`.

### Python Code


In [20]:
def chat_completion(user_input):

    # Generate embeddings from the user input
    user_embeddings = generate_embeddings(openai_client, user_input)
    
    # Query the chat history cache first to see if this question has been asked before
    cache_results = cache_search(vectors=user_embeddings , num_results=1)

    if len(cache_results) > 0:
        return cache_results[0]['completion'], True
    else:
        # Perform vector search on the movie collection
        print("\n New result\n")
        search_results = get_similar_docs(openai_client, dbsource, user_input, 3)
        print("\n search result done\n")

        print("Getting Chat History\n")
        # Chat history
        chat_history = get_chat_history(3)

        # Generate the completion
        print("Generating completions \n")
        completions_results = generatecompletionede(user_input, search_results, chat_history)

        print("Caching response \n")
        # Cache the response
        cacheresponse(user_input, user_embeddings, completions_results)

        print("\n")
        # Return the generated LLM completion
        return completions_results['choices'][0]['message']['content'], False

In [20]:
def generate_ad_image(ad_topic):
    daliClient = openai_client(
        api_key=openai_key
    )

    image_prompt = f'''
        Generate a photorealistic image of an ad campaign for selling {ad_topic}. 
        The image should be clean, with the item being sold in the foreground with an easily identifiable landmark of the city in the background.
        The image should also try to depict the weather of the location for the time of the year mentioned.
        The image should not have any generated text overlay.
    '''

    response = daliClient.images.generate(
        model=dalle,
        prompt= image_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
        )

    return response.data[0].url

Load some data like customer , or insurance sample 

In [40]:

createvectordb(colvector)
#loaddata(dbsource,'customer',"customer.json")
#ReadFeed('customer')
loaddata(dbsource,'insurance',"insurance.json")
ReadFeed('insurance')









vector
container created
Creating container...
Total count: 11
2024-09-19 10:54:55.733436
2024-09-18 10:54:55.733436
Generating embeddings for:  {"id": "77ED67C29F76283C314332FFB", "personame": "Darlleen Jenness", "birthdate": "1987-08-20", "customer_segment": "Mass Market", "marital_status": "divorce", "has_children": "false", "notes": "appartement", "customertype": "satisified", "monthly_avg_income": "6041 eur", "Claims": [{"ClaimsID": "78EFBF75C7", "Category": "fire", "amount": "492.07", "Date": "2023-02-27"}, {"ClaimsID": "E6022CFDB2", "Category": "health", "amount": "416.8", "Date": "2021-04-15"}, {"ClaimsID": "A6F3E1A96F", "Category": "house water", "amount": "-214.68", "Date": "2022-12-28"}, {"ClaimsID": "AECD6276C0", "Category": "carissues", "amount": "677.26", "Date": "2021-09-19"}, {"ClaimsID": "84E35344B9", "Category": "carissues", "amount": "743.11", "Date": "2022-04-06"}, {"ClaimsID": "2486C8EC09", "Category": "house water", "amount": "409.18", "Date": "2021-10-26"}, {"Cla

make a test to check 

In [21]:
get_similar_docs(openai_client,dbsource ,"groupama", 5)



Generating embeddings for:  groupama  with model:  text-embedding-ada-002


[{'id': 'e9eceeb5-35e6-4b5e-ba18-b2c3f58e2adc',
  'file': 'groupama',
  'text': "page_content='COMMUNIQUÉ  \nFINANCIER \n \nA propos du groupe Groupama  \nDepuis plus de 100 ans, le Groupe Groupama, fonde son action sur d es valeurs humanistes intemporelles pour permettre au plus grand no mbre de construire \nleur vie en confiance. Le Groupe Groupama, l’un des premiers groupes d’assurance mutualistes en France, dé veloppe ses activités  d’assurance, et de services \ndans dix pays. Le groupe compte 12 millions de sociétaires et clients  et 31 000 collaborateurs à travers le monde, avec un chiffre d’affaires an nuel de 15,5 \nmilliards d’euros.  Retrouvez toute l’actualité du Groupe Groupama sur son site internet (www.groupama.co m)  \n LE PRÉSENT COMMUNIQUÉ NE DOIT \nPAS ÊTRE DIFFUSÉ AUX ÉTATS-UNIS  \nParis, le 10 octobre  2022 \n \n \n \nGroupama Assurances Mutuelles a racheté pour 228,9 millions \nd’euros  les obligations senior subordonnées perpétuelles émises \nen 2014   \n \n \nGro

load some pdf from dataset sample

In [27]:

# Query 2
# Process a PDF and create embeddings
#file_path = "dataset\AXA_Conditions_Generales_Assurance_Habitation.pdf"  # Replace with your actual file path
#loadpdffile(dbsource,'pdf','axa',file_path)


#file_path = "dataset\Groupama-Resultats-offre-de-rachat-CP.pdf"  # Replace with your actual file path
#loadpdffile(dbsource,'pdf','groupama',file_path)
#ReadFeed('pdf')

#file_path = "dataset\\forvia.pdf"  # Replace with your actual file path
#loadpdffile(dbsource,'pdf','forvia',file_path)
#ReadFeed('pdf')


file_path = "dataset\\customers100.csv"  # Replace with your actual file path
loadcsvfile(dbsource,'csv','customers',file_path)
ReadFeed('csv')




Total count: 100
2024-09-19 17:38:20.492849
2024-09-18 17:38:20.492849
Generating embeddings for:  {"Index": "1", "Customer Id": "DD37Cf93aecA6Dc", "First Name": "Sheryl", "Last Name": "Baxter", "Company": "Rasmussen Group", "City": "East Leonard", "Country": "Chile", "Phone 1": "229.077.5154", "Phone 2": "397.884.0519x718", "Email": "zunigavanessa@smith.info", "Subscription Date": "2020-08-24", "Website": "http://www.stephenson.com/", "id": "5a3db567-1ad6-4d6a-9518-e8452cbf91ec", "file": "customers", "_rid": "ta5bAKFyqlkBAAAAAAAAAA==", "_self": "dbs/ta5bAA==/colls/ta5bAKFyqlk=/docs/ta5bAKFyqlkBAAAAAAAAAA==/", "_etag": "\"00005c0a-0000-0d00-0000-66ec45650000\"", "_attachments": "attachments/", "_ts": 1726760293, "_lsn": 2}  with model:  text-embedding-ada-002
5a3db567-1ad6-4d6a-9518-e8452cbf91ec
Generating embeddings for:  {"Index": "2", "Customer Id": "1Ef7b82A4CAAD10", "First Name": "Preston", "Last Name": "Lozano", "Company": "Vega-Gentry", "City": "East Jimmychester", "Country": "D

load some word sample

In [44]:
file_path = "dataset\sample.docx"  # Replace with your actual file path
loadwordfile(dbsource,'word','sample',file_path)


ReadFeed('word')

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\edeletang\AppData\Local\Temp\ipykernel_45640\763144153.py:1: SyntaxWarning: invalid escape sequence '\s'
  file_path = "dataset\sample.docx"  # Replace with your actual file path


page_content='OLSON HARRIS LTD.

Proposal for Services 



For Independent Contractors



The proposal for services is at the heart of an independent contractor’s sales tool kit. It combines key company information, your unique selling proposition, and your knowledge of a client’s needs and wants into a single document that can also serve as a basis for contract negotiations.

Use this sample proposal as an outline for developing your own.

To delete any shaded tips (like this one), just select the tip text and then press the spacebar.

The sample content throughout this proposal includes placeholder content for you to replace with your own.



Overview



Use the overview to provide a brief summary of the reason for the proposal, and how you can best support the client’s needs. You can expand on this summary throughout the rest of the proposal.' metadata={'source': 'dataset\\sample.docx'}
{'id': 'c2d0efd1-7740-47e7-9b88-49c8947108e9', 'file': 'sample', 'text': "page_content='OLSON HAR

can make test 

In [29]:
get_similar_docs(openai_client,dbsource ,"Joanna Bender",2)



Generating embeddings for:  Joanna Bender  with model:  text-embedding-ada-002


[{'Index': '5',
  'Customer Id': '053d585Ab6b3159',
  'First Name': 'Joanna',
  'Last Name': 'Bender',
  'Company': 'Martin, Lang and Andrade',
  'City': 'West Priscilla',
  'Country': 'Slovakia (Slovak Republic)',
  'Phone 1': '001-234-203-0635x76146',
  'Phone 2': '001-199-446-3860x3486',
  'Email': 'colinalvarado@miles.net',
  'Subscription Date': '2021-04-17',
  'Website': 'https://goodwin-ingram.com/',
  'id': 'bf870c75-10f5-4ff9-ad64-960d1b006eb6',
  'file': 'customers',
  '_rid': 'ta5bAKFyqlkFAAAAAAAAAA==',
  '_self': 'dbs/ta5bAA==/colls/ta5bAKFyqlk=/docs/ta5bAKFyqlkFAAAAAAAAAA==/',
  '_etag': '"0000600a-0000-0d00-0000-66ec45660000"',
  '_attachments': 'attachments/',
  '_ts': 1726760294}]

This code snippet is a part of a Jupyter Notebook that uses the `gradio` library to create a simple chatbot interface. Here's a detailed breakdown:

1. **Initialization**:
    - [`chat_history = []`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Initializes an empty list to store the chat history.

2. **Gradio Interface**:
    - [`with gr.Blocks() as demo:`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fgradio%2F__init__.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D ".venv/Lib/site-packages/gradio/__init__.py"): Creates a Gradio Blocks interface named [`demo`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A1%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").

3. **Components**:
    - [`chatbot = gr.Chatbot(label="Chat genereric with your data ")`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A6%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Creates a chatbot component with a label.
    - [`msg = gr.Textbox(label="Ask me about Your Data!")`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A10%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Creates a textbox for user input with a label.
    - [`clear = gr.Button("Clear")`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A11%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Creates a button to clear the chat.

4. **User Function**:
    - [`def user(user_message, chat_history, pk):`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Defines a function [`user`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") that takes a user message, chat history, and a primary key ([`pk`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A41%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")).
    - [`start_time = time.time()`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A15%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Starts a timer to measure the request time.
    - [`response_payload, cached = chat_completion(user_message)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A17%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Calls a function [`chat_completion`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X41sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to get the chatbot's response and whether it was cached.
    - [`end_time = time.time()`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A19%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Stops the timer.
    - [`elapsed_time = round((end_time - start_time) * 1000, 2)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A20%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Calculates the elapsed time in milliseconds.
    - [`response = response_payload`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A21%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Stores the response payload.
    - [`print(response_payload)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Fc%253A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.8.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Fbuiltins.pyi%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A1629%2C%22character%22%3A4%7D%5D "../../.vscode/extensions/ms-python.vscode-pylance-2024.8.1/dist/typeshed-fallback/stdlib/builtins.pyi"): Prints the response payload.
    - [`details = f"\n (Time: {elapsed_time}ms)"`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A24%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Creates a string with the elapsed time.
    - [`if cached: details += " (Cached)"`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A17%2C%22character%22%3A26%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Appends "(Cached)" to the details if the response was cached.
    - [`chat_history.append([user_message, response_payload + details])`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Appends the user message and response to the chat history.
    - [`return gr.update(value=""), chat_history`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fgradio%2F__init__.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D ".venv/Lib/site-packages/gradio/__init__.py"): Returns an updated Gradio component and the chat history.

5. **Event Handlers**:
    - [`msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A10%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Submits the user message to the [`user`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A13%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function when the textbox is submitted, updating the [`msg`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A10%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`chatbot`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A6%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") components.
    - [`clear.click(lambda: None, None, chatbot, queue=False)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X61sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A11%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): Clears the chatbot history when the clear button is clicked.

Overall, this code sets up a simple chatbot interface using Gradio, handles user input, and displays the chatbot's responses along with the time taken to generate them.

In [59]:
clearcache()

Container with id '<built-in function id>' created


In [45]:

from langchain_community.chat_message_histories.cosmos_db import CosmosDBChatMessageHistory

 

user = "test"
session = str(round(time.time() * 1000, 2))
cosmos_db_history = CosmosDBChatMessageHistory(
        cosmos_endpoint=HOST,
        cosmos_database=dbsource,
        cosmos_container=container_name,
        connection_string=cosmos_connection_string,
        session_id=session,
        user_id=user
)

cosmos_db_history.prepare_cosmos()

chat_history = []
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat genereric with your data ")
    
    
    
    msg = gr.Textbox(label="Ask me about Your  Data!")
    clear = gr.Button("Clear")

    def user(user_message, chat_history, pk):
        # Create a timer to measure the time it takes to complete the request
        start_time = time.time()
        # Get LLM completion
        response_payload, cached = chat_completion(user_message)
                
        cosmos_db_history.add_user_message(user_message)
        # Stop the timer
        end_time = time.time()
        elapsed_time = round((end_time - start_time) * 1000, 2)
        response = response_payload
        cosmos_db_history.add_ai_message(response)
        # Append user message and response to chat history
        details = f"\n (Time: {elapsed_time}ms)"
        if cached:
            details += " (Cached)"
        chat_history.append([user_message, response_payload + details])
        
        return gr.update(value=""), chat_history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)

    clear.click(lambda: None, None, chatbot, queue=False)

c:\Users\edeletang\Documents\demonosqlcomsosdb\.venv\Lib\site-packages\gradio\utils.py:986: UserWarning: Expected 3 arguments for function <function user at 0x000001DBAE717EC0>, received 2.
  warnings.warn(
c:\Users\edeletang\Documents\demonosqlcomsosdb\.venv\Lib\site-packages\gradio\utils.py:990: UserWarning: Expected at least 3 arguments for function <function user at 0x000001DBAE717EC0>, received 2.
  warnings.warn(


In [46]:
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\edeletang\Documents\demonosqlcomsosdb\.venv\Lib\site-packages\gradio\helpers.py:974: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


Generating embeddings for:  groupama  with model:  text-embedding-ada-002

 New result

Generating embeddings for:  groupama  with model:  text-embedding-ada-002

 search result done

Getting Chat History

Generating completions 

Prompt:  [{'role': 'system', 'content': '\n    You are an intelligent assistant for ypurdata . You are designed to provide helpful answers to user questions about  your data.\n    You are friendly, helpful, and informative and can be lighthearted. Be concise in your responses, but still friendly.\n        - Only answer questions related to the information provided below. \n        - Write two lines of whitespace between each answer in the list.\n    '}, {'role': 'user', 'content': 'groupama'}, {'role': 'system', 'content': "page_content='COMMUNIQUÉ  \nFINANCIER \n \nA propos du groupe Groupama  \nDepuis plus de 100 ans, le Groupe Groupama, fonde son action sur d es valeurs humanistes intemporelles pour permettre au plus grand no mbre de construire \nleur vie 

In [48]:
demo.close()

Closing server running on port: 7860
